# Putting the R(etrieval) in RAG

In [ ]:
%pip install pinecone openai bs4

  Using cached openai-1.102.0-py3-none-any.whl (812 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached jiter-0.10.0-cp310-cp310-macosx_11_0_arm64.whl (322 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached anyio-4.10.0-py3-none-any.whl (107 kB)
  Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
  Using cached beautifulsoup4-4.13.5-py3-none-any.whl (105 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
  Using cached h11-0.16.0-py3-none-any.whl (37 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)
  Using cached pydantic_core-2.33.2-cp310-cp310-macosx_11_0_arm64.whl (1.9 MB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached soupsieve-2.8-py3-none-any.whl (36 kB)

[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] T

In [ ]:
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import hashlib
from datetime import datetime, timezone

from tqdm import tqdm

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the Pinecone API key from user data
pinecone_key = os.getenv('PINECONE_API_KEY')

# Initialize the OpenAI client with the API key from user data
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

# Define constants for the Pinecone index, namespace, and engine
INDEX_NAME = 'semantic-search-rag'  # The name of the Pinecone index
NAMESPACE = 'default'  # The namespace to use within the index
ENGINE = 'text-embedding-3-small'  # The embedding model to use (vector size 1,536)

# Initialize the Pinecone client with the retrieved API key
pc = Pinecone(
    api_key=pinecone_key
)


In [6]:
# helper functions to get lists of embeddings from the OpenAI API
def get_embeddings(texts, engine=ENGINE):
    response = client.embeddings.create(
        input=texts,
        model=engine
    )

    return [d.embedding for d in list(response.data)]

def get_embedding(text, engine=ENGINE):
    return get_embeddings([text], engine)[0]

len(get_embedding('hi')), len(get_embeddings(['hi', 'hello']))

(1536, 2)

In [7]:
if INDEX_NAME not in pc.list_indexes().names():  # need to create the index
    print(f'Creating index {INDEX_NAME}')
    pc.create_index(
        name=INDEX_NAME,  # The name of the index
        dimension=1536,  # The dimensionality of the vectors for our OpenAI embedder
        metric='cosine',  # The similarity metric to use when searching the index
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Store the index as a variable
index = pc.Index(name=INDEX_NAME)
index

In [8]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'default': {'vector_count': 12}},
 'total_vector_count': 12,
 'vector_type': 'dense'}

In [9]:
def my_hash(s):
    # Return the MD5 hash of the input string as a hexadecimal string
    return hashlib.md5(s.encode()).hexdigest()

my_hash('I love to hash it')

'ae76cc4dfd345ecaeea9b8ba0d5c3437'

In [10]:
def prepare_for_pinecone(texts, engine=ENGINE, urls=None):
    # Get the current UTC date and time
    now =  datetime.now(timezone.utc).isoformat()

    # Generate vector embeddings for each string in the input list, using the specified engine
    embeddings = get_embeddings(texts, engine=engine)

    # Create tuples of (hash, embedding, metadata) for each input string and its corresponding vector embedding
    # The my_hash() function is used to generate a unique hash for each string, and the datetime.utcnow() function is used to generate the current UTC date and time
    responses = [
        (
            my_hash(text),  # A unique ID for each string, generated using the my_hash() function
            embedding,  # The vector embedding of the string
            dict(text=text, date_uploaded=now)  # A dictionary of metadata, including the original text and the current UTC date and time
        )
        for text, embedding in zip(texts, embeddings)  # Iterate over each input string and its corresponding vector embedding
    ]
    if urls and len(urls) == len(texts):
        for response, url in zip(responses, urls):
            response[-1]['url'] = url

    return responses


In [11]:
texts = ['hi']

In [12]:
_id, embedding, metadata = prepare_for_pinecone(texts)[0]

print('ID:  ',_id, '\nLEN: ', len(embedding), '\nMETA:', metadata)

ID:   49f68a5c8493ec2c0bf489821c21fc3b 
LEN:  1536 
META: {'text': 'hi', 'date_uploaded': '2025-08-31T10:07:30.727881+00:00'}


In [13]:
urls = ['fake.url']
_id, embedding, metadata = prepare_for_pinecone(texts, urls=urls)[0]

print('ID:  ',_id, '\nLEN: ', len(embedding), '\nMETA:', metadata)

ID:   49f68a5c8493ec2c0bf489821c21fc3b 
LEN:  1536 
META: {'text': 'hi', 'date_uploaded': '2025-08-31T10:07:31.773622+00:00', 'url': 'fake.url'}


In [14]:
def upload_texts_to_pinecone(texts, namespace=NAMESPACE, batch_size=None, show_progress_bar=False, urls=None):
    # Call the prepare_for_pinecone function to prepare the input texts for indexing
    total_upserted = 0
    if not batch_size:
        batch_size = len(texts)

    _range = range(0, len(texts), batch_size)
    for i in tqdm(_range) if show_progress_bar else _range:
        text_batch = texts[i: i + batch_size]
        if urls:
            url_batch = urls[i: i + batch_size]
            prepared_texts = prepare_for_pinecone(text_batch, urls=url_batch)
        else:
            prepared_texts = prepare_for_pinecone(text_batch)


        # Use the upsert() method of the index object to upload the prepared texts to Pinecone
        total_upserted += index.upsert(
            vectors=prepared_texts,
            namespace=namespace
        )['upserted_count']


    return total_upserted

In [15]:
# Call the upload_texts_to_pinecone() function with the input texts
upload_texts_to_pinecone(texts)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'default': {'vector_count': 12}},
 'total_vector_count': 12,
 'vector_type': 'dense'}

In [16]:
texts

['hi']

In [17]:
def query_from_pinecone(query, top_k=3, include_metadata=True):
    # get embedding from THE SAME embedder as the documents
    query_embedding = get_embedding(query, engine=ENGINE)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      namespace=NAMESPACE,
      include_metadata=include_metadata   # gets the metadata (dates, text, etc)
    ).get('matches')

In [18]:
# test that the index is empty
query_from_pinecone('hello')

[{'id': '888654645489d0899e8861bceec5707b',
  'metadata': {'date_uploaded': '2025-08-30T16:53:49.106914+00:00',
               'text': '\n'
                       '\n'
                       '\n'
                       '\n'
                       'You’re offline. This is a read only version of the '
                       'page.\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                       '\n'
                      

In [19]:
import hashlib

def delete_texts_from_pinecone(texts, namespace=NAMESPACE):
    # Compute the hash (id) for each text
    hashes = [hashlib.md5(text.encode()).hexdigest() for text in texts]

    # The ids parameter is used to specify the list of IDs (hashes) to delete
    return index.delete(ids=hashes, namespace=namespace)

# delete our text
delete_texts_from_pinecone(texts)


{}

In [20]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'default': {'vector_count': 12}},
 'total_vector_count': 12,
 'vector_type': 'dense'}

In [21]:
base_url = 'https://faq.ssa.gov'
medicare_faqs = base_url + '/en-US/topic?id=CAT-01092'
print(medicare_faqs)

from bs4 import BeautifulSoup
import requests

# get all links from medicare_faqs
urls = []
r = requests.get(medicare_faqs)
soup = BeautifulSoup(r.content, 'html.parser')
for link in soup.find_all('a'):
    if 'href' in link.attrs:
        if link['href'].startswith('/') and 'article' in link['href']:
            urls.append(base_url + link['href'])

urls

https://faq.ssa.gov/en-US/topic?id=CAT-01092


['https://faq.ssa.gov/en-us/Topic/article/KA-01735',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02166',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02154',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02131',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02713',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02125',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02113',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02137',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02148',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02983',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02989',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02995']

In [22]:
texts = []
for url in tqdm(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    body = soup.find('body').get_text()
    # CLEAN YOUR DATA HERE :)
    texts.append(body)

texts[0]

100%|██████████| 12/12 [00:13<00:00,  1.13s/it]


'\n\n\n\nYou’re offline. This is a read only version of the page.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n  \n\n\n\n\n\n\n\nProtect Yourself from Scams \n\n\n\n \n\n \n\n\n\n\nProtect Yourself from Scams\n\n\n\nSkip to main content Social Security Search  Menu  Español  Sign in\n\n\n\n\nFrequently Asked Questions\n\n\n\n\nLast Modified: \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFAQ Home\n\n\nTopics\n\n\r\n\t\t\t\t\tKA-01735\r\n\t\t\t\t\n\n\n\n\n\n Print\n\n\n\nHow do I get a replacement Medicare card? \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nViews: \n\n\n\nIf your Medicare card was lost, stolen, or destroyed, you can request a replacement online at Medicare.gov.\nYou can print an official copy of your card from your online Medicare account \nor call 1-800-MEDICARE (1-800-633-4227 TTY 1-877-486-2048) to order a replacement card to be sent in the mail.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFooter menu\n\n\n\n

In [23]:
BATCH_SIZE = 4
upload_texts_to_pinecone(texts, batch_size=BATCH_SIZE, urls=urls, show_progress_bar=True)

100%|██████████| 3/3 [00:02<00:00,  1.25it/s]


12

In [24]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'default': {'vector_count': 12}},
 'total_vector_count': 12,
 'vector_type': 'dense'}

In [25]:
results = query_from_pinecone('I lost my medicare card', top_k=3)
for result in results:
   print(result['metadata']['url'], result['score'], result['metadata']['text'][:50])

https://faq.ssa.gov/en-us/Topic/article/KA-01735 0.6935606 



You’re offline. This is a read only version of
https://faq.ssa.gov/en-us/Topic/article/KA-02713 0.520738661 



You’re offline. This is a read only version of
https://faq.ssa.gov/en-us/Topic/article/KA-02113 0.509479582 



You’re offline. This is a read only version of
